In [1]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

from keras.applications import vgg16
from keras import backend as K

Using TensorFlow backend.


In [2]:
base_image_path = 'base_image.jpg'              #path of base image
style_reference_image_path = 'picasso.jpg'       #path of the image whose style is to be used   
result_prefix = 'im'
iterations = 10

total_variation_weight = 1.0
style_weight = 0.7
content_weight = 0.5

width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [3]:
#returns a preprocessed image i.e mean centered
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# returns the image by changing the shape of the vector accordingly
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [5]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3)) 

input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0) #combination that'll be fed in model to get the features extracted.

In [6]:
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
for key in outputs_dict:
    print (key, outputs_dict[key])

block4_conv2 Tensor("block4_conv2/Relu:0", shape=(3, 50, 66, 512), dtype=float32)
block1_conv2 Tensor("block1_conv2/Relu:0", shape=(3, 400, 533, 64), dtype=float32)
block3_conv3 Tensor("block3_conv3/Relu:0", shape=(3, 100, 133, 256), dtype=float32)
block5_pool Tensor("block5_pool/MaxPool:0", shape=(3, 12, 16, 512), dtype=float32)
block5_conv3 Tensor("block5_conv3/Relu:0", shape=(3, 25, 33, 512), dtype=float32)
block4_conv3 Tensor("block4_conv3/Relu:0", shape=(3, 50, 66, 512), dtype=float32)
block3_pool Tensor("block3_pool/MaxPool:0", shape=(3, 50, 66, 256), dtype=float32)
input_1 Tensor("concat:0", shape=(3, 400, 533, 3), dtype=float32)
block5_conv2 Tensor("block5_conv2/Relu:0", shape=(3, 25, 33, 512), dtype=float32)
block2_pool Tensor("block2_pool/MaxPool:0", shape=(3, 100, 133, 128), dtype=float32)
block3_conv1 Tensor("block3_conv1/Relu:0", shape=(3, 100, 133, 256), dtype=float32)
block1_pool Tensor("block1_pool/MaxPool:0", shape=(3, 200, 266, 64), dtype=float32)
block4_conv1 Tensor(

In [10]:
#gram matrix of the feature representation of the images

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

#Calculating the style loss for combination of images and style image

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

#calculating the content loss of combination with respect to base image

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# Loss defined for smoothning of the image

def total_variation_loss(x):
    
    a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [11]:
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

In [12]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

In [13]:
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

loss += total_variation_weight * total_variation_loss(combination_image)

In [14]:
#calculating the total loss and combining into a function.
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [15]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [16]:
#encapsulating everythiong in to a class for loss and grad calculation, that will be used in f_min_lbfgs
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [19]:

x = np.zeros((1, img_nrows, img_ncols, 3))

for i in range(50):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    
    img = deprocess_image(x.copy())
    fname = 'result/im' + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 51351570000.0
Image saved as result/im_at_iteration_0.png
Iteration 0 completed in 114s
Start of iteration 1


/home/harsh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]


Current loss value: 24834697000.0
Image saved as result/im_at_iteration_1.png
Iteration 1 completed in 114s
Start of iteration 2
Current loss value: 19898247000.0
Image saved as result/im_at_iteration_2.png
Iteration 2 completed in 114s
Start of iteration 3
Current loss value: 17831797000.0
Image saved as result/im_at_iteration_3.png
Iteration 3 completed in 115s
Start of iteration 4
Current loss value: 16679174000.0
Image saved as result/im_at_iteration_4.png
Iteration 4 completed in 115s
Start of iteration 5
Current loss value: 15931288000.0
Image saved as result/im_at_iteration_5.png
Iteration 5 completed in 115s
Start of iteration 6
Current loss value: 15418546000.0
Image saved as result/im_at_iteration_6.png
Iteration 6 completed in 114s
Start of iteration 7
Current loss value: 15050583000.0
Image saved as result/im_at_iteration_7.png
Iteration 7 completed in 115s
Start of iteration 8
Current loss value: 14761970000.0
Image saved as result/im_at_iteration_8.png
Iteration 8 complet